In [24]:
import os

current_year = '1947'
journals_path = '~/Documents/Harry_Howard_Journal'
input_file = os.path.join(journals_path, f'{current_year}/{current_year}-entries.json')
embeddings_file = os.path.join(journals_path, 'journal-entries-with-embeddings.csv')
print(os.path.exists(embeddings_file))

False


In [19]:
import os
import pandas as pd
import tiktoken

from openai.embeddings_utils import get_embedding
import openai
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [20]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [21]:
# load & inspect dataset
df = pd.read_json(input_file)
df = df[['date', 'content']]
df = df.dropna()
df['combined'] = "Date: " + df['date'].astype(str).str.strip() + "; Text: " + df['content'].str.strip()

df.head(3)


,date,content,combined
0,1947-03-16,"Due to purposes I do not wish to speak about, ...",Date: 1947-03-16; Text: Due to purposes I do n...
1,1947-03-17,Eighteen years have passed since that memorabl...,Date: 1947-03-17; Text: Eighteen years have pa...
2,1947-03-19,"The weather has been most beautiful, just like...",Date: 1947-03-19; Text: The weather has been m...


In [22]:
encoding = tiktoken.get_encoding(embedding_encoding)

# omit reviews that are too long to embed
start_count = len(df)

df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens]

removed = start_count - len(df)
print(f'removed {removed}')

removed 0


In [25]:
df["embedding"] = df.combined.apply(lambda x: get_embedding(x, engine=embedding_model))
df.to_csv(embeddings_file, mode='a', header=False, index=False)    # append mode
